In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Model

#import tensorflow libraries 
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

from keras.models import Sequential

from sklearn.metrics import classification_report, confusion_matrix

from keras.layers import Layer
from keras import backend as K
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

In [2]:
df = pd.read_csv("data/cleaned_data1.csv")

In [3]:
X = df.drop(columns=["PCOS (Y/N)"])
y = df["PCOS (Y/N)"].values

# implementing train-test-split
train_features, test_features, train_target, test_target = train_test_split(X, y, test_size=0.33, random_state=13)

In [4]:
from sklearn.preprocessing import StandardScaler
S = StandardScaler()
train_features = S.fit_transform(train_features)
test_features = S.fit_transform(test_features)

Dimensionality Reduction using AutoEncoders

In [5]:
encoding_dim = 10

input_df = Input(shape=(37,))
encoded = Dense(encoding_dim, activation='relu')(input_df)
decoded = Dense(37, activation='relu')(encoded)

# encoder
autoencoder = Model(input_df, decoded)

# intermediate result
encoder = Model(input_df, encoded)

Apply ANN on reduced dimensional features

In [6]:
encoded_train_features = encoder.predict(train_features)
encoded_test_features = encoder.predict(test_features)

In [7]:
n_features = encoded_train_features.shape[1]
n_features

10

In [8]:
class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
#         print(input_shape)
#         print(self.units)
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff, 2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [9]:
model = Sequential()
model.add(Dense(10, input_dim=10, activation='relu'))
model.add(RBFLayer(10, 0.3))
model.add(Dense(1, activation='sigmoid'))

In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(encoded_train_features,train_target,validation_data=(encoded_test_features,test_target),epochs=100,batch_size=32)

Epoch 1/100
12/12 [==============================] - 0s 31ms/step - loss: 0.7244 - accuracy: 0.3333 - val_loss: 0.7097 - val_accuracy: 0.4000
Epoch 2/100
12/12 [==============================] - 0s 5ms/step - loss: 0.7039 - accuracy: 0.4576 - val_loss: 0.6942 - val_accuracy: 0.5086
Epoch 3/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6898 - accuracy: 0.6158 - val_loss: 0.6846 - val_accuracy: 0.6400
Epoch 4/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6789 - accuracy: 0.6638 - val_loss: 0.6751 - val_accuracy: 0.6571
Epoch 5/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6688 - accuracy: 0.6836 - val_loss: 0.6671 - val_accuracy: 0.6571
Epoch 6/100
12/12 [==============================] - 0s 5ms/step - loss: 0.6597 - accuracy: 0.6836 - val_loss: 0.6600 - val_accuracy: 0.6571
Epoch 7/100
12/12 [==============================] - 0s 4ms/step - loss: 0.6520 - accuracy: 0.6836 - val_loss: 0.6534 - val_accuracy: 0.6571
Epoch 8/100


Epoch 59/100
12/12 [==============================] - 0s 3ms/step - loss: 0.5106 - accuracy: 0.6836 - val_loss: 0.5740 - val_accuracy: 0.6571
Epoch 60/100
12/12 [==============================] - 0s 3ms/step - loss: 0.5098 - accuracy: 0.6836 - val_loss: 0.5743 - val_accuracy: 0.6571
Epoch 61/100
12/12 [==============================] - 0s 3ms/step - loss: 0.5087 - accuracy: 0.6836 - val_loss: 0.5734 - val_accuracy: 0.6571
Epoch 62/100
12/12 [==============================] - 0s 3ms/step - loss: 0.5074 - accuracy: 0.6836 - val_loss: 0.5731 - val_accuracy: 0.6571
Epoch 63/100
12/12 [==============================] - 0s 3ms/step - loss: 0.5067 - accuracy: 0.6836 - val_loss: 0.5728 - val_accuracy: 0.6571
Epoch 64/100
12/12 [==============================] - 0s 3ms/step - loss: 0.5057 - accuracy: 0.6836 - val_loss: 0.5715 - val_accuracy: 0.6571
Epoch 65/100
12/12 [==============================] - 0s 3ms/step - loss: 0.5048 - accuracy: 0.7006 - val_loss: 0.5713 - val_accuracy: 0.6743
Epoch 

In [11]:
loss, acc = model.evaluate(encoded_test_features, test_target, verbose=0)
print('Test Accuracy: %.3f' %(acc * 100.0))

Test Accuracy: 70.857
